## Extract trails
* This notebook will concatenate all epochs assoicated with a trail in to one data point.
* The goal of this is to try to describe data as congruent or incongruent.
    * In other words a trail will be described as congruent or incongruent

In [51]:
%run ../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
%run ../preprocessing/StimCodes.ipynb
import os

import mne
from mne.preprocessing import ICA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (50.0, 30.0)
matplotlib.rcParams.update({'font.size': 15})

from sklearn.preprocessing import StandardScaler, normalize

In [84]:
path = 'E:\EpochedEEG'
epoch_files = os.listdir(path)
file = os.path.join(path,epoch_files[0])
epoch = mne.read_epochs(file, preload = True)

Reading E:\EpochedEEG\20131216_1441_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
5424 matching events found
Applying baseline correction (mode: mean)
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [85]:
epoch = update_stim_code_event_ids(epoch)

In [86]:
epoch.event_id

{'trail_start': 31,
 'left_button': 131,
 'right_button': 132,
 'congruent': 133,
 'incongruent': 134,
 'correct': 129,
 'incorrect': 130,
 't_baby': 135,
 't_bell': 136,
 't_bird': 137,
 't_burp': 138,
 't_dog': 139,
 't_drum': 140,
 't_knock': 141,
 't_laugh': 142,
 't_phone': 143,
 't_train': 144,
 't_water': 145,
 'f_baby': 155,
 'f_bell': 156,
 'f_bird': 157,
 'f_burp': 158,
 'f_dog': 159,
 'f_drum': 160,
 'f_knock': 161,
 'f_laugh': 162,
 'f_phone': 163,
 'f_train': 164,
 'f_water': 165,
 'b-f2-wrd': 1,
 'b-f1-wrd': 2,
 'b-tg-wrd': 3,
 'b-f2-spk': 4,
 'b-f1-spk': 5,
 'b-tg-spk': 6,
 'b-f2-pic': 7,
 'b-f1-pic': 8,
 'b-tg-pic': 9,
 'b-f2-snd': 10,
 'b-f1-snd': 11,
 'b-tg-snd': 12,
 'f-f2-wrd': 13,
 'f-f1-wrd': 14,
 'f-tg-wrd': 15,
 'f-f2-spk': 16,
 'f-f1-spk': 17,
 'f-tg-spk': 18,
 'f-f2-pic': 19,
 'f-f1-pic': 20,
 'f-tg-pic': 21,
 'f-f2-snd': 22,
 'f-f1-snd': 23,
 'f-tg-snd': 24}

In [87]:
events = epoch.events[:,-1]
event_ids = epoch.event_id
print("len events:", len(events))

len events: 5424


In [88]:
epoch.get_data().shape

(5424, 129, 180)

In [89]:
#combining events of a trail into 1 list
trials = []
event_trial = []
for event in events:
    if event == 31 and event_trial != []:
        trials.append(event_trial)
        event_trial = []
        event_trial.append(event)
    else:
        event_trial.append(event)
# print(np.array(trails))

In [91]:
trials_and_events = []
trial_dict = {}
for trial in trials:
    for event in trial:
        for key, value in event_ids.items():
            if value == event:
                trial_dict[key] = event
    trials_and_events.append(trial_dict)
    trial_dict = {}
trials_and_events[1]

{'trail_start': 31,
 't_train': 144,
 'f_laugh': 162,
 'b-f1-pic': 8,
 'b-tg-wrd': 3,
 'b-f2-pic': 7,
 'right_button': 132,
 'incongruent': 134,
 'incorrect': 130}

In [36]:
#remove dead trail at the beginning of study
del trials_and_events[0]

###  Combine epochs by trail (event_ids are combined, but combining epochs is a different task)

In [20]:
epoch.get_data().shape

(5424, 129, 180)

### combine epochs and give them a label equal to Audio or Visual 

In [64]:
# creating a new dictionary where each trail will equal one of the values in the dictinary Mode_lexi that is defined in 
# ../preprocessing/StimCodes.ipynb
trial_event_ids = []
for trial in trials_and_events:
    #check individual stim-codes and match them with corresponding combination defined in stimcodes.ipynb
    stim_combination = []
    
    for key in trial:
#         print(trial[key])
        if int(trial[key]) >= 1 and int(trial[key]) <= 24:
            stim_combination.append(trial[key])
#             print(stim_combination)
        if len(stim_combination) == 3: # stim-code combination complete, it can now be matched to a value in Mode_lexi
            stim_combination_tuple = (stim_combination[0],stim_combination[1],stim_combination[2])
#             print(stim_combination_tuple)
            
            for stim_combo_key in modal_lexi:
                stim_combo1 = modal_lexi[stim_combo_key][0]
                stim_combo2 = modal_lexi[stim_combo_key][1]

                if stim_combination_tuple == stim_combo1 or stim_combination == stim_combo2:
                    dict = {stim_combo_key : trial}      
                    trial_event_ids.append(dict)
            break;
 

In [66]:
trial_event_ids[0] #this trial has stim-code combination VVLN, which is a visual stimuli 

{'VVLN': {'trail_start': 31,
  't_train': 144,
  'f_laugh': 162,
  'b-f1-pic': 8,
  'b-tg-wrd': 3,
  'b-f2-pic': 7,
  'right_button': 132,
  'incongruent': 134,
  'incorrect': 130}}

In [71]:
updated_epochs = epochs[1:]#dropped first epoch - it is a dead epoch
trial_epochs = []

for epoch in updated_epochs:
    2
epoch.get_data()[:2,:,:].shape

(2, 129, 180)

In [96]:
#combine epochs into trials with index
trials = []
event_trial = []
start_index = 0
current_index = 0

for i in range(len(events)):
    if events[i] == 31 and event_trial != []:
        
        trials.append(((start_index, current_index),event_trial))
        
        event_trial = []
        start_index+=1
        event_trial.append(events[i])
    else:
        current_index+=1
        event_trial.append(events[i])

In [97]:
trials

[((0, 1), [131]),
 ((1, 9), [31, 144, 162, 8, 3, 7, 132, 134, 130]),
 ((2, 17), [31, 142, 162, 8, 3, 7, 132, 133, 129]),
 ((3, 24), [31, 142, 162, 8, 3, 7, 132, 133]),
 ((4, 32), [31, 142, 162, 8, 3, 7, 132, 133, 129]),
 ((5, 39), [31, 144, 162, 8, 3, 7, 131, 134]),
 ((6, 46), [31, 142, 162, 8, 3, 7, 131, 133]),
 ((7, 53), [31, 144, 162, 8, 3, 7, 131, 134]),
 ((8, 60), [31, 142, 162, 8, 3, 7, 132, 133]),
 ((9, 67), [31, 144, 162, 8, 3, 7, 131, 134]),
 ((10, 74), [31, 144, 162, 8, 3, 131, 7, 134]),
 ((11, 81), [31, 144, 162, 8, 3, 131, 7, 134]),
 ((12, 89), [31, 142, 162, 8, 3, 7, 132, 133, 129]),
 ((13, 97), [31, 142, 162, 8, 3, 7, 132, 133, 129]),
 ((14, 105), [31, 142, 162, 8, 3, 7, 132, 133, 129]),
 ((15, 113), [31, 144, 162, 8, 3, 131, 7, 134, 129]),
 ((16, 120), [31, 142, 162, 8, 3, 132, 7, 133]),
 ((17, 128), [31, 144, 162, 8, 3, 7, 131, 134, 129]),
 ((18, 135), [31, 142, 162, 8, 3, 7, 132, 133]),
 ((19, 142), [31, 144, 162, 8, 3, 7, 131, 134]),
 ((20, 150), [31, 144, 162, 8, 3, 

array([ 31, 144, 162,   8,   3,   7, 132, 134])